# PuLP basics

FONTE: 

- https://www.youtube.com/watch?v=qNA7Y57bXLQ&list=PLr5INjIG1eXXdQuv5wZwreUyStGKsQIff

- https://scipbook.readthedocs.io/en/latest/flp.html#capacitated-facility-location-problem

**Instalando/carregando o módulo**

In [77]:
from pulp import *

**Definindo os conjuntos**

In [78]:
FOODS = ['OATMEAL', 'CHICKEN', 'EGGS', 'MILK']
CUSTUMER = [1, 2, 3, 4, 5]
FACILITY = ['FAC 1', 'FAC 2', 'FAC 3']

**Parâmetros**

In [79]:
dem = {1 : 80,
       2 : 270,
       3 : 250,
       4 : 160,
       5 : 170}

In [80]:
actcost = {'FAC 1' : 1000,
           'FAC 2' : 1000,
           'FAC 3' : 1000}

In [81]:
maxam = {'FAC 1' : 500,
         'FAC 2' : 500,
         'FAC 3' : 500,}

In [82]:
transp = {'FAC 1' : {1:4, 2:5, 3:6, 4:8, 5:10},
          'FAC 2' : {1:6, 2:4, 3:3, 4:5, 5:8},
          'FAC 3' : {1:9, 2:7, 3:4, 4:3, 5:4}}

**Definindo o problema**

In [83]:
prob = LpProblem("FacilityLocation", LpMinimize)

**Definindo as variáveis de decisão**

In [84]:
use_vars = LpVariable.dicts("UseLocation", FACILITY, 0, 1, LpBinary)

In [85]:
serv_vars = LpVariable.dicts("Service", [(i,j) for i in CUSTUMER
                                          for j in FACILITY],
                      0) #OBS: definindo uso de 'j' para FACILITY e 'i' para CUSTUMER

In [86]:
foods_vars = LpVariable.dicts("food", FOODS, 0)

**Função objetivo**

In [87]:
prob += lpSum(actcost[j]*use_vars[j] for j in FACILITY) \
        + lpSum(transp[j][i]*serv_vars[(i,j)] for j in FACILITY
                                              for i in CUSTUMER)

#OBS: usar atentamente os indices j e i.

**Constraints**

In [88]:
for i in CUSTUMER:
    prob += lpSum(serv_vars[(i,j)] for j in FACILITY) == dem[i]

In [89]:
for j in FACILITY:
    prob += lpSum(serv_vars[(i,j)] for i in CUSTUMER) <= maxam[j]

In [90]:
for i in CUSTUMER:
    for j in FACILITY:
        prob += serv_vars[(i,j)] <= dem[i]*use_vars[j]

**Solving**

In [91]:
prob.solve()

1

In [92]:
print('Status:', LpStatus[prob.status])

Status: Optimal


In [97]:
TOL = 0.000001
for i in FACILITY:
    if use_vars[i].varValue > TOL:
        print('Estabilish facility at site ', i, '.', sep='')

Estabilish facility at site FAC 2.
Estabilish facility at site FAC 3.


In [94]:
for v in prob.variables():
    print(v.name,'=',v.varValue)

Service_(1,_'FAC_1') = 0.0
Service_(1,_'FAC_2') = 80.0
Service_(1,_'FAC_3') = 0.0
Service_(2,_'FAC_1') = 0.0
Service_(2,_'FAC_2') = 270.0
Service_(2,_'FAC_3') = 0.0
Service_(3,_'FAC_1') = 0.0
Service_(3,_'FAC_2') = 150.0
Service_(3,_'FAC_3') = 100.0
Service_(4,_'FAC_1') = 0.0
Service_(4,_'FAC_2') = 0.0
Service_(4,_'FAC_3') = 160.0
Service_(5,_'FAC_1') = 0.0
Service_(5,_'FAC_2') = 0.0
Service_(5,_'FAC_3') = 170.0
UseLocation_FAC_1 = 0.0
UseLocation_FAC_2 = 1.0
UseLocation_FAC_3 = 1.0


In [95]:
print('Total cost of food/serving', value(prob.objective))

Total cost of food/serving 5570.0
